<a href="https://colab.research.google.com/github/Ali-MH-Mansour/Document-classification/blob/main/Classification_doc_Vdhya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Connect to the Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Functions for the data set preprocessing

In [ ]:
import re
import string
import pandas as pd
import numpy as np
import math
from numpy import array 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from sklearn.cluster import KMeans
from collections import Counter 
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
#-------------------------------------------------------------------------------
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
#-------------------------------------------------------------------------------
def preprocess_text(text):
    text = re.sub(r'\d+' , '', text)
    text = remove_punctuation(text)
    text = text.lower()
    text = text.strip()
    text = re.sub(r'\b\w{1,3}\b', '', text)
    tokens = toknizing(text)
    pos_map = {'J': 'a', 'N': 'n', 'R': 'r', 'V': 'v'}
    pos_tags = pos_tag(tokens)
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(t.lower(), pos=pos_map.get(p[0], 'v')) for t, p in pos_tags]
    return tokens

#--------------------------------------------------------------------------------------------
def remove_punctuation(text):
     punctuations = '''!()-[]{};:'"\,<>./?@#$%^+&*_~'''
     no_punct = ""
     for char in text:
        if char not in punctuations:
            no_punct = no_punct + char
     return no_punct

#--------------------------------------------------------------------------------------------
def toknizing(text):
  stop_words = set(stopwords.words('english'))
  tokens = word_tokenize(text)
  result = [i for i in tokens if not i in stop_words]
  return result
#-------------------------------------------------------------------------------
def cosine_distance(u, v):
  return np.dot(u, v) / (math.sqrt(np.dot(u, u)) * math.sqrt(np.dot(v, v)))
#-------------------------------------------------------------------------------
!pip install soyclustering
from soyclustering import SphericalKMeans
from scipy.sparse import csr_matrix
#-------------------------------------------------------------------------------
def spherical_kmeans(n_clusters_, list_of_words_vectors,sim_digree):
  embeddings_matrix_csr = csr_matrix(list_of_words_vectors)
  spherical_kmeans = SphericalKMeans( max_similar=sim_digree, init='similar_cut', 
      n_clusters = n_clusters_)
  labels = spherical_kmeans.fit_predict(embeddings_matrix_csr)
  centers = spherical_kmeans.cluster_centers_
  return labels , centers

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


##Data sets Downolad

In [ ]:
import pandas as pd
RE_file_path="/content/drive/...../Data/reuters-text.csv"
data_DF = pd.read_csv(RE_file_path)
# rename columns
data_DF.columns.values[0] = "class"
data_DF.columns.values[1] = "document"
data_DF.head()


In [ ]:
BBC_file_path="/content/drive/MyDrive/PHD/Data/Reuters88.csv"
data_DF = pd.read_csv(BBC_file_path)

# rename columns
data_DF.columns.values[0] = "class"
data_DF.columns.values[1] = "document"

data_DF.head()

,class,document
0,0,ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...
1,1,CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...
2,2,JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...
3,3,THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...
4,1,SRI LANKA GETS USDA APPROVAL FOR WHEAT PRICE\n...


In [ ]:
# Factorize class names to numbers
data_DF['class'] = pd.factorize(data_DF['class'])[0]
data_DF['document'] = data_DF.document.apply(lambda x: ' '.join(preprocess_text(x)))
data_DF.head()

,class,document
0,0,asian exporter fear damage usjapan rift mount ...
1,1,china daily say vermin grain stock survey prov...
2,2,japan revise longterm energy demand downwards ...
3,3,thai trade deficit widen first quarter thailan...
4,1,lanka get usda approval wheat price food depar...


## Train embedding model on Reuters data set

In [ ]:
# Train fast text model on your dataset
import gensim
from gensim.models import FastText, Word2Vec


In [ ]:
documents_as_list = data_DF['document'].tolist()
doc_as_tokens =[]
for doc in documents_as_list:
  doc_as_tokens.append(doc.split())

In [ ]:
model_fasttext = FastText(doc_as_tokens, size=300, window=15, min_count=5, workers=4,sg=1)

In [ ]:
#model_fasttext.save("path/......../word2vec.model")

In [ ]:
model_w2v = Word2Vec(doc_as_tokens, size=300, window=15, min_count=5, workers=4,sg=1)

## Create Dictionary of words & embedd doc

In [ ]:
dic_w_v = {}
all_doc_emb = []
for doc in doc_as_tokens:
  doc_emb = []
  for w in doc:
    try :
      emb = model_fasttext.wv[w]
      doc_emb.append(emb)
      dic_w_v.update({w : emb})
    except: 
      continue;
  all_doc_emb.append(doc_emb) # doc as list of embedded words

#Averaged embeddings

In [ ]:
avg_fasttext =[]
dzero=[0] * 300
i=0
for d in all_doc_emb:
  l=len(d)
  s=sum(d)
  if l!=0:
    av=s/l
    avg_fasttext.append(av)
  else:
    avg_fasttext.append(dzero)
  i=i+1

#BOC

## Clustering the words (concepts extraction)

In [ ]:
list_of_words_vectors = list(dic_w_v.values())

In [ ]:
# number of clusters
K_clusters= 300
sim_digree=0.3
######### Clustering
labels , centers = spherical_kmeans(K_clusters , list_of_words_vectors, sim_digree)
print("The clusters details", Counter(labels), "\n")

The clusters details Counter({105: 298, 112: 295, 198: 287, 58: 249, 7: 233, 128: 232, 176: 213, 142: 211, 11: 204, 253: 196, 145: 196, 82: 194, 22: 187, 123: 186, 277: 185, 178: 183, 191: 181, 201: 180, 33: 180, 180: 180, 259: 178, 288: 177, 140: 177, 27: 176, 287: 175, 31: 170, 282: 168, 234: 163, 227: 161, 48: 161, 10: 160, 138: 160, 222: 160, 86: 159, 279: 158, 249: 154, 43: 154, 286: 154, 98: 151, 118: 151, 89: 150, 267: 150, 146: 149, 245: 148, 199: 147, 163: 147, 171: 146, 40: 146, 242: 146, 187: 146, 61: 145, 19: 143, 200: 143, 254: 143, 75: 141, 283: 140, 204: 139, 77: 136, 9: 136, 46: 134, 229: 133, 216: 130, 52: 129, 103: 128, 210: 128, 226: 126, 281: 126, 272: 125, 34: 125, 126: 124, 44: 123, 96: 123, 14: 122, 291: 120, 72: 119, 93: 118, 144: 116, 275: 115, 293: 115, 177: 114, 238: 111, 239: 111, 141: 110, 15: 110, 45: 110, 189: 109, 155: 108, 218: 108, 207: 108, 134: 107, 196: 107, 109: 106, 251: 106, 299: 106, 69: 104, 113: 103, 124: 101, 292: 101, 101: 100, 166: 99, 190:

# Bag of weighted Concepts

### Documents vectorization

In [ ]:
#خاص بحقيبة الكلمات الاصلية
def BOC(embeddeings_document,alpha,centers):
  concept_occurence = 0
  ## intilize doc array
  CF_vector = np.zeros(len(centers)).tolist()
  for cnt in range(len(centers)):
    for emb_word in embeddeings_document:
      Wsim= cosine_distance(emb_word , centers[cnt])
      if Wsim  > alpha:
        concept_occurence = concept_occurence + 1 #for tf
        CF_vector[cnt] = CF_vector[cnt] + 1;
      else:
        continue;    
  if concept_occurence !=0:
    for i in CF_vector:
      i = i / concept_occurence  #count/
  return CF_vector
#-------------------------------------------------------------
def CF_IDF_BOC(documents_embeddings , alpha , centers):
  CFs = []
  counter = 0
  Df = np.zeros(len(centers)).tolist() # بطول العناقيد
  for doc in documents_embeddings: 
    CF = BOC(doc , alpha , centers )
    for i in CF: 
      if i > 0:
        Df[counter] = Df[counter] + 1 #  i[M-1]
      counter = counter+1
    counter = 0
    CFs.append(CF)
  #########  calculate IDF
  IDF=[]
  for df in Df:
    if df!=0:
      N=len(documents_embeddings)
      IDF.append( math.log10( (N+1) /df ) )
    else:
      IDF.append(0)  
  #Calculate Cf-IDF
  for i in range(N):
    l=len(CFs[i])
    for j in range(l):
       CFs[i][j] = CFs[i][j] * IDF[j]  #cf*idf
  return CFs

In [ ]:
# Thershold similarity values
alpha = 0.4

final_embeddings  = CF_IDF_BOC(all_doc_emb , alpha , centers)

#doc2vec

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(documents_as_list)]
model_doc2vec = Doc2Vec(documents, vector_size=300, window=10, min_count=5, workers=4 , epochs=15)

In [ ]:
doc2vec_embeddings = model_doc2vec.docvecs

#TFidf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Building a TF IDF matrix out of the corpus of reviews
vectorizer = TfidfVectorizer(sublinear_tf=False, use_idf=True, max_df=0.5,  stop_words='english')
tfidf_vector = vectorizer.fit_transform(documents_as_list)

In [ ]:
print(tfidf_vector.todense()[0])

[[0. 0. 0. ... 0. 0. 0.]]


#Bag-of-words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer( max_df=0.5,  stop_words='english')
bow_vector = vectorizer.fit_transform(documents_as_list)

In [ ]:
bow_vector.todense()

matrix([[0, 0, 0, ..., 0, 0, 0]])

##Train SVM for dataset calssification

In [ ]:
final_embeddings = tfidf_vector
#final_embeddings = bow_vector
#final_embeddings = doc2vec_embeddings
#final_embeddings = avg_fasttext
# final_embeddings = boc_vector

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

from sklearn.metrics import f1_score

#-------------------------------------------------------------------------------
classes = data_DF['class'].tolist()
x_train , x_test , y_train , y_test = train_test_split(final_embeddings , classes , test_size = 0.30 , random_state = 0)
svm =  LinearSVC( random_state= 44).fit(x_train , y_train)

##**Evaluation**

In [ ]:
y_pred=svm.predict(x_test)
F1 = f1_score(y_test,y_pred, average="macro")
print("The micro average-weighted f1 score = ", F1)

The micro average-weighted f1 score =  0.9011425924993196


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9372056514913658

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  32,    5,    2,    1,   24,    4,   19,    3],
       [  12,   27,    4,    2,   39,    1,   11,    2],
       [   2,    5,   49,    0,   65,    2,   47,    5],
       [   6,    5,    4,    5,   29,    2,    6,    1],
       [   4,    5,    9,    0,  505,    8,  128,    2],
       [   2,    0,    4,    1,   32,   64,   25,    8],
       [   6,    3,    5,    1,  151,    7, 1046,    0],
       [   6,    1,    2,    0,   44,   22,   10,   26]])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.36      0.51       117
           1       0.51      0.57      0.53       152
           2       0.61      0.90      0.73       177
           3       0.84      0.82      0.83       109
           4       0.61      0.44      0.51       113

    accuracy                           0.64       668
   macro avg       0.68      0.62      0.62       668
weighted avg       0.67      0.64      0.62       668

